In [166]:
from os.path import join, dirname
import math
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import SVD
# SVD = singular value decomposition

In [1]:
from moviebox.recommender import recommender

In [2]:
movieId = 32

In [3]:
movieId

32

In [4]:
recommendationsNumber = 10

In [10]:
showPlots = False

In [11]:
interactive = True

In [12]:
recommender(
    movieID=movieId,
    recommendationsNumber=recommendationsNumber,
    showPlots=showPlots,
    interactive=interactive
)

✔ Imported Data		0.2s
✔ Split Data		0.0s
✔ Trained Engine	6.0s
✔ Found Similarities	1.3s
❯ Given Movie
• Title: Zindagi - ID: 32
  Categories: Bollywood

★ Top 10 Recommendations
• Title: Kedi - ID: 3083
  Categories: Action

• Title: Bawarchi - ID: 3404
  Categories: Musical, Bollywood, World cinema

• Title: I Love You - ID: 926
  Categories: Romance Film, Drama

• Title: Vaastav: The Reality - ID: 3500
  Categories: Musical, World cinema, Bollywood

• Title: Lajja - ID: 4621
  Categories: Musical, Drama, Bollywood, World cinema

• Title: Baana Kaathadi - ID: 976
  Categories: Romance Film

• Title: Chetna - ID: 3455
  Categories: Action

• Title: Charas - ID: 1188
  Categories: Thriller, Action

• Title: Aaptamitra - ID: 2123
  Categories: Thriller, Horror, World cinema

• Title: Veera Bahu - ID: 3023
  Categories: Action



0

In [13]:
recommender

<function moviebox.recommender.recommender(movieID=2874, recommendationsNumber=3, showPlots=False, interactive=False)>

In [32]:
csvPath = pd.read_csv('/home/rahul/Workspace/spograde-recommendation-engine/moviebox/moviebox/dataset/movies.csv')

In [169]:
csvPath

,ID,Title,Category,Plot
0,0,Ghosts of Mars,"""Thriller"", ""Science Fiction"", ""Horror"", ""A...","Set in the second half of the 22nd century, th..."
1,1,White Of The Eye,"""Thriller"", ""Erotic thriller"", ""Psychologica...",A series of murders of rich young women throug...
2,2,A Woman in Flames,"""Drama""","Eva, an upper class housewife, becomes frustra..."
3,3,The Sorcerer's Apprentice,"""Family Film"", ""Fantasy"", ""Adventure"", ""Wor...","Every hundred years, the evil Morgana returns..."
4,4,Little city,"""Romantic comedy"", ""Ensemble Film"", ""Comedy-...","Adam, a San Francisco-based artist who works a..."
...,...,...,...,...
4995,4995,Morgan and Destiny's Eleventeenth Date: The Ze...,"""Short Film"", ""Romance Film"", ""Animation""",Morgan and Destiny pay a visit to an ornate ...
4996,4996,Indradhanura Chhai,"""Drama""",The film looks at the lives of three women liv...
4997,4997,Barfuß bis zum Hals,"""Comedy film""",A nudist site in eastern Germany is sold to Di...
4998,4998,Donovan's Reef,"""Romantic comedy"", ""Action/Adventure"", ""Come...","The film begins with Thomas ""Boats"" Gilhooley ..."


In [42]:
#values => return queryset containing dictionaries
#flatten() => json nested form
#tolist() => return the list of values
titles = csvPath[['Title']].values.flatten().tolist()

In [43]:
categories = csvPath[['Category']].values.flatten().tolist()

In [44]:
plot = csvPath[['Plot']].values.flatten().tolist()

In [54]:
dataset = {
    'titles': titles,
    'categories': categories,
    'plots': plot
}

In [55]:
def validateMovieID(movieId):
    if not (isinstance(movieId, int) and movieId >= 0 and movieId <= 4999):
        print('Invalid value for movieID {}'.format(movieId))
    return 0

In [56]:
validateMovieID(9000)

Invalid value for movieID 9000


0

In [57]:
{
    'title': dataset['titles'][32],
    'category': dataset['categories'][32]
}

{'title': 'Zindagi', 'category': '"Bollywood"'}

In [61]:
list_zip = []
for i in range(20):
    list={
        'title': dataset['titles'][i],
        'category': dataset['categories'][i]
    }
    list_zip.append(list)
print(list_zip)

[{'title': 'Ghosts of Mars', 'category': '"Thriller",  "Science Fiction",  "Horror",  "Adventure",  "Supernatural",  "Action",  "Space western"'}, {'title': 'White Of The Eye', 'category': '"Thriller",  "Erotic thriller",  "Psychological thriller"'}, {'title': 'A Woman in Flames', 'category': '"Drama"'}, {'title': "The Sorcerer's Apprentice", 'category': '"Family Film",  "Fantasy",  "Adventure",  "World cinema"'}, {'title': 'Little city', 'category': '"Romantic comedy",  "Ensemble Film",  "Comedy-drama",  "Drama",  "Comedy",  "Romantic drama",  "Romance Film"'}, {'title': 'Henry V', 'category': '"Costume drama",  "War film",  "Epic",  "Period piece",  "Drama",  "Film adaptation"'}, {'title': 'Aaah Belinda', 'category': '"Comedy"'}, {'title': 'The Mechanical Monsters', 'category': '"Science Fiction",  "Adventure",  "Animation",  "Short Film",  "Fantasy",  "Family Film",  "Action"'}, {'title': 'Mary Poppins', 'category': '"Children\'s/Family",  "Musical",  "Fantasy",  "Comedy",  "Drama",

In [64]:
TfidfVectorizer

sklearn.feature_extraction.text.TfidfVectorizer

In [176]:
vectorizer = TfidfVectorizer(
    lowercase=True,
    min_df=3,
    max_df=0.9,
    ngram_range=(1,2),
    stop_words='english'
)

In [177]:
plotsTFIDF = vectorizer.fit_transform(dataset['plots'])

In [178]:
cosineSimilarities = linear_kernel(plotsTFIDF, plotsTFIDF)

In [179]:
cosineSimilarities[32]

array([0.00468561, 0.00674292, 0.01851156, ..., 0.02557757, 0.01698776,
       0.0062504 ])

In [186]:
scores = enumerate(cosineSimilarities[32])

In [187]:
sortedScores = sorted(scores, key=lambda x: x[1], reverse=True)

In [188]:
sortedScores

[(32, 1.0000000000000002),
 (3083, 0.32555371611786676),
 (3404, 0.32533001257003635),
 (926, 0.27735422104847135),
 (3500, 0.2530637645150256),
 (4621, 0.22560146184244376),
 (976, 0.2239951292730065),
 (3455, 0.1856528863616968),
 (1188, 0.14630077302488192),
 (2123, 0.14314981403989874),
 (3023, 0.13107228483618968),
 (3564, 0.1244118819204614),
 (3847, 0.11507636312089714),
 (141, 0.09920925319057185),
 (3260, 0.09911781554727644),
 (4535, 0.09197395886745656),
 (1677, 0.08538077784712696),
 (609, 0.08490706840450489),
 (48, 0.08428986758846103),
 (10, 0.07360064790835517),
 (721, 0.07000846238961417),
 (416, 0.0689239021017538),
 (2379, 0.0687073650286783),
 (487, 0.06707766964141063),
 (361, 0.06547653584918924),
 (1596, 0.06537526723694626),
 (1142, 0.06466875342437876),
 (2853, 0.06190700655211801),
 (4547, 0.06142853958058211),
 (3175, 0.06066205997804313),
 (2734, 0.0598596372137708),
 (2143, 0.05881883403626826),
 (611, 0.058817837953700276),
 (2433, 0.05870744566117169),
 (

In [189]:
#5 => no. of recommendations
movie_Recommendations = sortedScores[1:5+1]

In [190]:
dataset['titles']

['Ghosts of Mars',
 'White Of The Eye',
 'A Woman in Flames',
 "The Sorcerer's Apprentice",
 'Little city',
 'Henry V',
 'Aaah Belinda',
 'The Mechanical Monsters',
 'Mary Poppins',
 'White on Rice',
 'Anbu Thozhi',
 'Baby Boy',
 'Bindiya Chamkegi',
 'Vandanam',
 'Anokha Rishta',
 'Karayilekku Oru Kadal Dooram',
 'Siam Sunset',
 'Kausthubham',
 'Archie: To Riverdale and Back Again',
 'Troops',
 'Daddy and Them',
 'The Gods Must Be Crazy',
 'Rudo y Cursi',
 'Kinjite: Forbidden Subjects',
 'The Great New Wonderful',
 'Loverboy',
 'Chandra Mukhi',
 'Juarez',
 'Chasing Ghosts: Beyond the Arcade',
 'It Came Upon the Midnight Clear',
 'The Human Tornado',
 "Adventurer's Fate",
 'Zindagi',
 'Terrorama',
 'One Small Step: The Story of the Space Chimps',
 'The Little Hut',
 'Nariman',
 'Aftershock: Beyond the Civil War',
 'Drums in the Deep South',
 'Closing the Ring',
 'The Boston Strangler',
 'Deadly Voyage',
 'The Hero: Love Story of a Spy',
 "Donald's Crime",
 '33 Scenes from Life',
 'Herma

In [191]:

for movie in movie_Recommendations:
    a = {
        'movie_id': movie[0],
        'title': dataset['titles'][movie[0]],
        'category': dataset['categories'][movie[0]],
        'matching': math.ceil(movie[1]*100)
    }
    print(a)

{'movie_id': 3083, 'title': 'Kedi', 'category': '"Action"', 'matching': 33}
{'movie_id': 3404, 'title': 'Bawarchi', 'category': '"Musical",  "Bollywood",  "World cinema"', 'matching': 33}
{'movie_id': 926, 'title': 'I Love You', 'category': '"Romance Film",  "Drama"', 'matching': 28}
{'movie_id': 3500, 'title': 'Vaastav: The Reality', 'category': '"Musical",  "World cinema",  "Bollywood"', 'matching': 26}
{'movie_id': 4621, 'title': 'Lajja', 'category': '"Musical",  "Drama",  "Bollywood",  "World cinema"', 'matching': 23}
